In [8]:

#%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, callbacks

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Helper libraries
import numpy as np
import pandas as pd
import scipy.io as sio
from sklearn.model_selection import train_test_split
from sklearn.utils import compute_class_weight
import math

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from pathlib import Path
import os.path
#import datetime, os

from keras.applications import VGG16, VGG19, ResNet50, InceptionV3, InceptionResNetV2, Xception, MobileNet, DenseNet121, NASNetMobile, EfficientNetB0, MobileNetV2, MobileNetV3Large
from tensorflow.python.keras.layers import Conv2D, Flatten, Dropout, Dense, MaxPooling2D
from keras.models import load_model
from keras.utils.vis_utils import plot_model
from tensorflow.keras.models import model_from_json


Num GPUs Available:  1


## Data Splitting ##

In [9]:
# df = pd.read_csv(r'machine-learning-in-science-ii-2023/training_norm.csv')
# df['filename'] = df["image_id"].astype(str) + ".png"
# df = df.sample(frac=1).reset_index(drop=True)
# df.to_csv("training_norm_shuffle.csv", encoding='utf-8', index=False)

df = pd.read_csv(r'machine-learning-in-science-ii-2023/training_norm_shuffle.csv')
print(df)

x_train = df[0:int(len(df) * 0.6)]
x_validate = df[int(len(df) * 0.6):int(len(df) * 0.8)]
x_evaluate = df[int(len(df) * 0.8):]

# x_train = df[0:int(len(df) * 0.8)]
# x_validate = df[int(len(df) * 0.8):]


       image_id   angle  speed   filename
0           414  0.8125    1.0    414.png
1         17532  0.6875    1.0  17532.png
2         16480  0.3125    0.0  16480.png
3          6450  0.6875    0.0   6450.png
4         12314  0.6250    0.0  12314.png
...         ...     ...    ...        ...
14821      9844  0.6875    1.0   9844.png
14822     12367  0.5000    1.0  12367.png
14823       343  0.9375    1.0    343.png
14824      3838  0.6875    0.0   3838.png
14825     13301  0.5000    1.0  13301.png

[14826 rows x 4 columns]


## Data Preprocessing ##

In [10]:

batch_size = 32
img_size = [240, 320]
shift = 0.1

training_datagen = ImageDataGenerator(
    rescale = 1./255,
    fill_mode='nearest',
    width_shift_range=shift,
    height_shift_range=shift
)

speed_train_generator = training_datagen.flow_from_dataframe(
    dataframe=x_train,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col="speed",
    target_size=img_size,
    batch_size=batch_size,
    shuffle=False,
    class_mode='other')

angle_train_generator = training_datagen.flow_from_dataframe(
    dataframe=x_train,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col="angle",
    target_size=img_size,
    batch_size=batch_size,
    shuffle=False,
    class_mode='other')

validation_datagen = ImageDataGenerator(rescale = 1./255)

speed_val_generator = validation_datagen.flow_from_dataframe(
    dataframe=x_validate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col="speed",
    target_size=img_size,
    shuffle=False,
    class_mode='other')

angle_val_generator = validation_datagen.flow_from_dataframe(
    dataframe=x_validate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col="angle",
    target_size=img_size,
    shuffle=False,
    class_mode='other')

evaluate_datagen = ImageDataGenerator(rescale = 1./255)

speed_eval_generator = evaluate_datagen.flow_from_dataframe(
    dataframe=x_evaluate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col="speed",
    target_size=img_size,
    shuffle=False,
    class_mode='other')

angle_eval_generator = evaluate_datagen.flow_from_dataframe(
    dataframe=x_evaluate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col="angle",
    target_size=img_size,
    shuffle=False,
    class_mode='other')



Found 8895 validated image filenames.
Found 8895 validated image filenames.
Found 2965 validated image filenames.
Found 2965 validated image filenames.
Found 2966 validated image filenames.
Found 2966 validated image filenames.


In [11]:
filename = os.listdir("machine-learning-in-science-ii-2023/test_data/test_data") 

df = pd.DataFrame(filename)
df.columns = ["filename"]

df[['file', 'type']] = df.filename.str.split(".", expand = True)
df["file"] = df["file"].astype(str).astype(int)

df.sort_values(by=['file'], inplace=True)
df.reset_index(drop=True, inplace=True)
print(df)
submission_datagen = ImageDataGenerator(rescale = 1./255)

submission_images = submission_datagen.flow_from_dataframe(
    dataframe=df,
    directory="machine-learning-in-science-ii-2023/test_data/test_data",
    x_col="filename",
    target_size=img_size,
    shuffle=False,
    class_mode=None)

      filename  file type
0        1.png     1  png
1        2.png     2  png
2        3.png     3  png
3        4.png     4  png
4        5.png     5  png
...        ...   ...  ...
1015  1016.png  1016  png
1016  1017.png  1017  png
1017  1018.png  1018  png
1018  1019.png  1019  png
1019  1020.png  1020  png

[1020 rows x 3 columns]
Found 1020 validated image filenames.


## Transfer Learning ##

In [30]:
shape = (*img_size, 3) # inherited image size with 3 color filters
input_shape = [240, 320, 3]
transfer = MobileNetV2(
    input_shape=input_shape, 
    include_top=False, 
    weights='imagenet',
    classifier_activation="relu"
)

for layer in transfer.layers[:-2]:
    layer.trainable = False

## Model Creation and Evaluation ##

In [31]:
# Speed CNN base
speed_model = models.Sequential()

speed_model.add(transfer)
speed_model.build()
        
speed_model.add(layers.Conv2D(1280, (3, 3), strides=(2,2), activation='relu'))
speed_model.add(layers.BatchNormalization())

speed_model.add(layers.Flatten()),

speed_model.add(layers.Dense(128, activation='relu'))
speed_model.add(layers.BatchNormalization())
speed_model.add(layers.Dropout(0.2))

speed_model.add(layers.Dense(64, activation='relu'))
speed_model.add(layers.BatchNormalization())
speed_model.add(layers.Dropout(0.2))

# Output layer
speed_model.add(layers.Flatten())
speed_model.add(layers.Dense(1,  activation='sigmoid'))

In [32]:
# Angle CNN base
angle_model = models.Sequential()

angle_model.add(transfer)
angle_model.build()
        
angle_model.add(layers.Conv2D(128, (3, 3), strides=(2,2), activation='relu'))
angle_model.add(layers.BatchNormalization())

angle_model.add(layers.Flatten()),

# model_1.add(layers.Dense(128, activation='relu'))
angle_model.add(layers.Dense(128, activation='relu'))
angle_model.add(layers.BatchNormalization())
angle_model.add(layers.Dropout(0.2))

# Output layer
angle_model.add(layers.Flatten())
angle_model.add(layers.Dense(1,  activation='relu'))

In [33]:
speed_model.build()
#angle_model.build()

In [34]:
speed_model.summary()
#angle_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 8, 10, 1280)      2257984   
 ional)                                                          
                                                                 
 conv2d_4 (Conv2D)           (None, 3, 4, 1280)        14746880  
                                                                 
 batch_normalization_10 (Bat  (None, 3, 4, 1280)       5120      
 chNormalization)                                                
                                                                 
 flatten_8 (Flatten)         (None, 15360)             0         
                                                                 
 dense_10 (Dense)            (None, 128)               1966208   
                                                                 
 batch_normalization_11 (Bat  (None, 128)             

In [35]:
speed_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.0001),
    loss='mean_squared_error'
)

# angle_model.compile(
#     optimizer=tf.optimizers.Adam(learning_rate=0.001),
#     loss='mean_squared_error'
# )

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=20,
    verbose=1,
    mode='min',
    restore_best_weights=True
)

In [36]:
history = speed_model.fit(
    speed_train_generator,
    batch_size=batch_size,
    validation_data=speed_val_generator,
    callbacks=[es],
    epochs=100
)

Epoch 1/100
278/278 [==============================] - 112s 389ms/step - loss: 0.0576 - val_loss: 0.0222
Epoch 2/100
278/278 [==============================] - 109s 393ms/step - loss: 0.0269 - val_loss: 0.0199
Epoch 3/100
278/278 [==============================] - 109s 392ms/step - loss: 0.0195 - val_loss: 0.0155
Epoch 4/100
278/278 [==============================] - 106s 380ms/step - loss: 0.0163 - val_loss: 0.0159
Epoch 5/100
278/278 [==============================] - 113s 406ms/step - loss: 0.0145 - val_loss: 0.0162
Epoch 6/100
278/278 [==============================] - 114s 410ms/step - loss: 0.0126 - val_loss: 0.0154
Epoch 7/100
278/278 [==============================] - 114s 411ms/step - loss: 0.0116 - val_loss: 0.0152
Epoch 8/100
278/278 [==============================] - 117s 419ms/step - loss: 0.0109 - val_loss: 0.0159
Epoch 9/100
278/278 [==============================] - 108s 389ms/step - loss: 0.0096 - val_loss: 0.0161
Epoch 10/100
278/278 [==============================] -

In [37]:
test_loss = speed_model.evaluate(
    speed_eval_generator,
    verbose=1
)

93/93 [==============================] - 9s 92ms/step - loss: 0.0138


In [38]:
history = angle_model.fit(
    angle_train_generator,
    batch_size=batch_size,
    validation_data=angle_val_generator,
    callbacks=[es],
    epochs=100
)

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [ ]:
test_loss = angle_model.evaluate(
    angle_eval_generator,
    verbose=1
)

93/93 [==============================] - 8s 83ms/step - loss: 0.0066


In [67]:
speed_model.save("speed-model.h5")
#angle_model.save("angle-model1.h5")

## Model Prediction ##

In [ ]:
angle_prediction = angle_model.predict(submission_images)
speed_prediction = speed_model.predict(submission_images)

speed_prediction += 0.5
speed_prediction = np.floor(speed_prediction)

submission = {"angle":angle_prediction , "speed":speed_prediction}

df = pd.DataFrame(submission)
df.index += 1 
df.to_csv('Submission.csv', index_label="image_id")


32/32 [==============================] - 3s 69ms/step


In [ ]:
diff = abs(x_evaluate["angle"] - angle_prediction)
results = pd.DataFrame({"Train":x_evaluate["angle"], "Predicted":angle_prediction, "Difference":diff})
results

,Train,Predicted,Difference
11860,0.6875,0.686453,0.001047
11861,0.7500,0.751384,0.001384
11862,0.6250,0.693253,0.068253
11863,0.6875,0.623532,0.063968
11864,0.6875,0.730291,0.042791
...,...,...,...
14821,0.6875,0.708553,0.021053
14822,0.5000,0.433915,0.066085
14823,0.9375,0.777668,0.159832
14824,0.6875,0.794277,0.106777


In [ ]:
results = pd.DataFrame({"Train":x_evaluate["speed"],"Predicted":speed_prediction})
#results

## Fine-tuning ##

In [ ]:
speed_model = load_model("speed-model.h5")
angle_model = load_model("angle-model1.h5")


angle_transfer = angle_model.layers[0]
angle_transfer.trainable = True

for layer in angle_transfer.layers[:-25]:
    layer.trainable = False
        
angle_model.summary()


speed_transfer = speed_model.layers[0]
speed_transfer.trainable = True

for layer in speed_transfer.layers[:-25]:
    layer.trainable = False
    
speed_model.summary()


In [ ]:

speed_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.00001),
    loss='mean_squared_error'
)

angle_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.00001),
    loss='mean_squared_error'
)

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.0005,
    patience=30,
    verbose=1,
    mode='min',
    restore_best_weights=True
)


In [ ]:
history = angle_model.fit(
    angle_train_generator,
    batch_size=batch_size,
    validation_data=angle_val_generator,
    callbacks=[es],
    epochs=300
)

In [ ]:
test_loss = angle_model.evaluate(
    angle_eval_generator,
    verbose=1
)

history = speed_model.fit(
    speed_train_generator,
    batch_size=batch_size,
    validation_data=speed_val_generator,
    callbacks=[es],
    epochs=300
)

In [ ]:
from sklearn.metrics import classification_report

test_loss = speed_model.evaluate(
    speed_eval_generator,
    verbose=1
)


In [ ]:
speed_model.save("speed-model-fine-tuning.h5")
angle_model.save("angle-model-fine-tuning.h5")

In [ ]:
angle_prediction = angle_model.predict(submission_images)
speed_prediction = speed_model.predict(submission_images)

speed_prediction += 0.5
speed_prediction = np.floor(speed_prediction)

submission = {"angle":angle_prediction , "speed":speed_prediction}

df = pd.DataFrame(submission)
df.index += 1 
df.to_csv('Submission-fine-tuning.csv', index_label="image_id")

In [ ]:
model = load_model("model-fine-tuning.hdf5")
plot_model(model, show_layer_names=True, show_shapes=True,show_layer_activations=True)

## Clean Image File ##

In [ ]:
speed_model = load_model("speed-model.h5")
angle_model = load_model("angle-model.h5")

model = load_model("model.h5")

x_all = pd.read_csv(r'machine-learning-in-science-ii-2023/training_norm.csv')
x_all['filename'] = x_all["image_id"].astype(str) + ".png"

clean_datagen = ImageDataGenerator(rescale = 1./255)

clean_generator = clean_datagen.flow_from_dataframe(
    dataframe=x_all,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    target_size=img_size,
    shuffle=False,
    class_mode=None)

angle_prediction = angle_model.predict(clean_generator)
speed_prediction = speed_model.predict(clean_generator)

# predict = model.predict(clean_generator)

# angle_prediction = predict[:,0]
# speed_prediction = predict[:,1]

diff = abs(x_all["angle"] - angle_prediction)

speed_prediction += 0.5
speed_prediction = np.floor(speed_prediction)

correct = (x_all["speed"] == speed_prediction)

clean_data = {
    "image_id":x_all["image_id"], 
    "Actual Angle":x_all["angle"], 
    "Predicted Angle":angle_prediction, 
    "Angle Difference":diff, 
    "Actual Speed":x_all["speed"], 
    "Predicted Speed":speed_prediction,
    "Correct?":correct
}

df = pd.DataFrame(clean_data)
df.index += 1 
df.to_csv('clean-results.csv', index_label="image_id")


Found 14826 validated image filenames.


2023-03-28 16:03:45.536609: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-03-28 16:03:47.001088: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: Permission denied


  4/464 [..............................] - ETA: 25s   

2023-03-28 16:03:48.709861: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


464/464 [==============================] - 46s 76ms/step


## Run other models ##

In [12]:
angle_model = load_model("angle-model.h5")
speed_model = load_model("speed-model.h5")

speed_prediction = speed_model.predict(submission_images)
angle_prediction = angle_model.predict(submission_images)

speed_prediction = speed_prediction[0]

speed_prediction = speed_prediction.flatten()
angle_prediction = angle_prediction.flatten()

speed_prediction += 0.5
speed_prediction = np.floor(speed_prediction)

submission = {"angle":angle_prediction , "speed":speed_prediction}

df = pd.DataFrame(submission)
df.index += 1 
df.to_csv('other-submission.csv', index_label="image_id")

2023-03-29 00:04:33.541672: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-03-29 00:04:35.216262: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: Permission denied


 3/32 [=>............................] - ETA: 1s  

2023-03-29 00:04:37.229596: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


32/32 [==============================] - 4s 91ms/step


In [ ]:
speed_model = load_model("sakthi-model.hdf5")

test_loss = speed_model.evaluate(
    speed_eval_generator,
    verbose=1
)